Project : CamemBERT  
Unit : Advanced Machine Learning   
MSc. Intelligent systems engineering  
SORBONNE UNIVERSITÉ  

--- Students ---  
@SSivanesan - Shivamshan SIVANESAN  
@Emirtas7 - Emir TAS  
@KishanthanKingston - Kishanthan KINGSTON  
code inspiré de https://huggingface.co/blog/how-to-train

In [1]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

In [3]:
file_path = "fr_part_1.txt"

with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()

paths = [str(content)]

In [5]:
paths = [file_path]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=32_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [6]:
tokenizer.save_model(".", "camembert")

['./camembert-vocab.json', './camembert-merges.txt']

In [7]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./camembert-vocab.json",
    "./camembert-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

print(
    tokenizer.encode("Je m'appelle Kishanthan.")
)

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [9]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, file_path: str, evaluate: bool = False):
        tokenizer = ByteLevelBPETokenizer(
            "./camembert-vocab.json",
            "./camembert-merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)

        self.examples = []

        # Read text from your file
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.read().splitlines()

        self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

# Specify the path to your dataset file (fr_part_1.txt)
dataset_file_path = "fr_part_1.txt"

# Create an instance of your dataset
my_dataset = MyDataset(file_path=dataset_file_path)


In [10]:
from transformers import RobertaForMaskedLM, RobertaTokenizer, AdamW

In [11]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import time
from torch.utils.data import DataLoader
from transformers import AdamW

dataloader = DataLoader(my_dataset, batch_size=1, shuffle=True)

optimizer = AdamW(model.parameters(), lr=1e-3)

# Vérification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Boucle d'entraînement
num_epochs = 100

total_time_start = time.time()
for epoch in range(num_epochs):
    total_loss = 0.0  # Initialisez la perte totale pour cette epoch

    for batch in dataloader:
        model.train()
        
        # Envoyer le batch sur le GPU si disponible
        batch = batch.to(device)

        # Convertir les identifiants de tokens en texte
        texts = tokenizer.batch_decode(batch.tolist(), skip_special_tokens=True)

        # Préparer les inputs pour RoBERTa (ajouter les masques de remplissage)
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

        # Envoyer les inputs sur le GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # C'est pour calculer les logits du modèle
        outputs = model(**inputs, labels=inputs["input_ids"])

        # Nous calculons ici la perte
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch}, Loss: {loss.item():.4f}")

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {average_loss:.4f}")

total_time_end = time.time()
print("Temps total mis pour l'entraînement du modèle :", total_time_end - total_time_start)
    
# Nous sauvegardons notre modèle et notre Tokenizer
model.save_pretrained("CamemBERT_MLM")
tokenizer.save_pretrained("CamemBERT_Tokenizer")


Epoch 1/100, Batch tensor([[    0,  1924,   307,   952, 13081,   263,   338,   293,  5063, 14163,
            16,   303,  3064,  6044,   373,  8334,  5948,   272,   293,  2756,
           291,   272,   293,  7694, 11915,   272,   293,  1499,    18,   330,
           298,   325,  2441,   272,   597, 17455,   291,   336, 24597, 13414,
            16,  3545,    16, 23939,  2533,   272,   560,   307,  2879,    16,
          2336,  1325,    16, 11218,   272,  3064,    16,  8793,   272,   264,
           298,  2734,    16,  9518,    16,  9186,   474,   502,  3911,   338,
           303,   952,   272,  6296,    16,   302,   298,   325,  9946,   264,
           298,  3835,   291,   264,   298, 10130,   261,   298,   289,  2070,
           367,   564,   624,  2312,  9208,   344,  3064,  1707,     2]],
       device='cuda:0'), Loss: 7.3770
Epoch 1/100, Batch tensor([[    0,  2085, 17890,  1383,   344,  4060,   399, 28740,   303, 25922,
          6345,   597,  1833,   392,   511, 12002,   293,  2

In [ ]:
# On charge notre modèle et le tokenizer pré-entraînés
loaded_model = RobertaForMaskedLM.from_pretrained("CamemBERT_MLM")
loaded_tokenizer = RobertaTokenizer.from_pretrained("CamemBERT_Tokenizer")

# On évalue notre modèle
loaded_model.eval()

# Un exemple de texte
text_to_test = "J'ai raté mon examen donc je vais <mask>."

# Préparer les inputs pour RoBERTa (ajouter les masques de remplissage)
inputs = loaded_tokenizer(text_to_test, return_tensors="pt", padding=True, truncation=True)

# Envoyer les inputs sur le GPU si disponible
inputs = {k: v.to(device) for k, v in inputs.items()}

# Prédictions
with torch.no_grad():
    predictions = loaded_model(**inputs)

predicted_ids = torch.argmax(predictions.logits, dim=-1)
predicted_tokens = [loaded_tokenizer.decode(token.item()) for token in predicted_ids[0]]
print("Texte prédit :", " ".join(predicted_tokens))
